In [1]:

from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_f0d6ce325e0f4bc08812229b8d429dbe(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', '9a0cc60102244d368e96a83f25d4ca89')
    hconf.set(prefix + '.username', '0caf8026c98a4342ac027a05416e6dee')
    hconf.set(prefix + '.password', 'D[Cvr1bgf9DM^I{C')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_f0d6ce325e0f4bc08812229b8d429dbe(name)

spark = SparkSession.builder.getOrCreate()

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df_data_1 = spark.read.json('swift://CursoSpark.' + name + '/funcionarios.json')
# df_data_1.take(5)


In [2]:
funcDF = df_data_1

In [3]:
funcDF.show()

+------+-----+----------------+-------+----+
|deptid|idade|            nome|salario|sexo|
+------+-----+----------------+-------+----+
|  1000|   42|   Josias Rebelo|   7000|   m|
|  2000|   50|Mauricio Gonheim|   9500|   m|
|  1000|   36| Bruno Velasquez|   6700|   m|
|  1000|   41|  Ananda Tavares|   9300|   f|
|  2000|   34|     Carlos Maia|   5500|   m|
+------+-----+----------------+-------+----+



In [4]:
funcDF.printSchema()

root
 |-- deptid: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- salario: string (nullable = true)
 |-- sexo: string (nullable = true)



In [5]:
type(funcDF)

pyspark.sql.dataframe.DataFrame

In [6]:
funcDF.select("nome").show()

+----------------+
|            nome|
+----------------+
|   Josias Rebelo|
|Mauricio Gonheim|
| Bruno Velasquez|
|  Ananda Tavares|
|     Carlos Maia|
+----------------+



In [8]:
funcDF.filter(funcDF["idade"]=="50").show()

+------+-----+----------------+-------+----+
|deptid|idade|            nome|salario|sexo|
+------+-----+----------------+-------+----+
|  2000|   50|Mauricio Gonheim|   9500|   m|
+------+-----+----------------+-------+----+



In [11]:
funcDF.groupBy("sexo").count().show()

+----+-----+
|sexo|count|
+----+-----+
|   m|    4|
|   f|    1|
+----+-----+



In [14]:
funcDF.groupBy("deptid").agg({"salario":"avg","idade":"max"}).show()

+------+----------+-----------------+
|deptid|max(idade)|     avg(salario)|
+------+----------+-----------------+
|  2000|        50|           7500.0|
|  1000|        42|7666.666666666667|
+------+----------+-----------------+



In [15]:
# Registrando uma tabela temporaria
funcDF.createOrReplaceTempView("funcionario")

In [17]:
spark.sql("Select deptid, max(idade),avg(salario) from funcionario group by deptid").show()

+------+----------+----------------------------+
|deptid|max(idade)|avg(CAST(salario AS DOUBLE))|
+------+----------+----------------------------+
|  2000|        50|                      7500.0|
|  1000|        42|           7666.666666666667|
+------+----------+----------------------------+



In [18]:
# Registrando o dataframe como temp table
funcDF.createOrReplaceTempView("funcTB")

In [20]:
spark.sql("select * from funcTB where salario > 6000").show()

+------+-----+----------------+-------+----+
|deptid|idade|            nome|salario|sexo|
+------+-----+----------------+-------+----+
|  1000|   42|   Josias Rebelo|   7000|   m|
|  2000|   50|Mauricio Gonheim|   9500|   m|
|  1000|   36| Bruno Velasquez|   6700|   m|
|  1000|   41|  Ananda Tavares|   9300|   f|
+------+-----+----------------+-------+----+



In [21]:
#Persistindo a temp table
funcTB3 = spark.table("funcTB")

In [22]:
type(funcTB3)

pyspark.sql.dataframe.DataFrame

In [25]:
funcTB3.filter("idade='42'").show()

+------+-----+-------------+-------+----+
|deptid|idade|         nome|salario|sexo|
+------+-----+-------------+-------+----+
|  1000|   42|Josias Rebelo|   7000|   m|
+------+-----+-------------+-------+----+

